In [1]:
cd /content/drive/My Drive/Colab Notebooks/tobigs

/content/drive/My Drive/Colab Notebooks/tobigs


In [2]:
ls

 kaggle.json             test_df.csv                    torch_submission.csv
 pytorch                 test_df.csv.zip                train_df.csv
 sample_submission.csv  'Tobigs_Kannada digits.ipynb'   train_df.csv.zip


In [0]:
import pandas as pd
import numpy as np

In [0]:
sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [5]:
X = train.iloc[:, 1:].values / 255
y = train.iloc[:, 0].values
X_test = test.iloc[:, 1:].values / 255
print(X.shape, y.shape, X_test.shape)

(42000, 784) (42000,) (18000, 784)


In [0]:
X = X.reshape(-1,1,  28, 28)
X_test = X_test.reshape(-1,1, 28, 28)

### pytorch 로 모델을 만들면서 느낀점(주관적) + 정리
- 일단 keras 보다 조금 불편했다.
- Convolution layer에 집어넣기 위해 keras 에서 x를 reshape를 해주어야한다. (첨에 이걸 안해줬더니 오류남ㅠ_ㅠ) 그리고 이거 reshape 순서 때문에 오류가 날수 있으므로 주의

In [0]:
from torch.utils.data import Dataset, DataLoader # torch.utils.data.Dataset를 상속해야 한다

class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform  # 지정하면 이미지 데이터에 어떤 변형을 가할지를 transform function의 묶음(Compose)로 전달한다.
    
    def __len__(self):  # dataset의 전체 개수를 알려준다.
        return len(self.X)
    
    def __getitem__(self, idx):  # parameter로 idx를 넘겨주면 idx번째의 데이터를 반환한다.
        X = self.X[idx]
        y = self.y[idx]
        return X, y

class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]   # test 는 y값이 없는것을 확인할수 있음
        return X

In [0]:
traindataloader = DataLoader(TrainDataset(X, y), batch_size=128, shuffle=True, num_workers=4)   
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4)

- num_worker :  데이터 프로세싱에 필요한 코어 개수를 할당해준느것 . 적당한 개수를 정해주면 된다. 

1. activation function : relu
2. batchnormalization
3. max pooling
4. optimization : Adam
5. regularization : drop out


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):  # nn.Module은 모든 PyTorch 모델의 base class이다.
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2) # conv1
        self.conv1_bn = nn.BatchNorm2d(num_features=32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2) # conv2
        self.conv2_bn = nn.BatchNorm2d(num_features=64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels= 128, kernel_size=3, stride=1, padding=1) # conv3
        self.conv3_bn = nn.BatchNorm2d(num_features=128)
        
        self.fc1 = nn.Linear(in_features=128*6*6, out_features=1024) # linear 1
        self.fc1_bn = nn.BatchNorm1d(num_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=512) # linear 2
        self.fc2_bn = nn.BatchNorm1d(num_features=512)
        self.fc3 = nn.Linear(in_features=512, out_features=256) # linear 3
        self.fc3_bn = nn.BatchNorm1d(num_features=256)
        self.fc4 = nn.Linear(in_features=256, out_features=64) # linear 4
        self.fc4_bn = nn.BatchNorm1d(num_features=64)
        self.out = nn.Linear(in_features=64, out_features=10) # output
    
    def forward(self, t):

        t = F.relu(self.conv1_bn(self.conv1(t)))
        t = F.max_pool2d(t, kernel_size=2, stride=2) # (1, 14, 14)
        
        t = F.relu(self.conv2_bn(self.conv2(t)))
        t = F.max_pool2d(t, kernel_size=2, stride=2) # (1, 7, 7)
        
        t = F.relu(self.conv3_bn(self.conv3(t)))
        t = F.max_pool2d(t, kernel_size=2, stride=1) # (1, 6, 6)
        
        t = F.relu(self.fc1_bn(self.fc1(t.reshape(-1, 128*6*6))))
        t = F.relu(self.fc2_bn(self.fc2(t)))
        t = F.relu(self.fc3_bn(self.fc3(t)))
        t = F.relu(self.fc4_bn(self.fc4(t)))
        t = self.out(t)
        
        return t

net = Net()

In [10]:
net

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4608, out_features=1024, bias=True)
  (fc1_bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc2_bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc3_bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Lin

# Train


In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()   # loss function 
optimizer = optim.Adam(net.parameters(), lr=2e-3, weight_decay=1e-3) # 최적화 함수 지정

cuda(device=None): 모든 model parameter를 GPU 버퍼에 옮기는 것으로 GPU를 쓰고 싶다면 이를 활성화해주어야 한다. .cuda()는 optimizer를 설정하기 전에 실행되어야 한다. 잊어버리지 않으려면 모델을 생성하자마자 쓰는 것이 좋다.

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [13]:
# put the model on device (hopefully a GPU!)
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4608, out_features=1024, bias=True)
  (fc1_bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc2_bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc3_bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Lin

## train 의 기본 구조 
1. output = model(data) : 모델에서 정의한 forward 단계실행. 모델에 data를 전달하여 예상하는 label 값 계산
2. loss = criterion(output, label) : 모델에서 나온 output과 label을 이용해 loss 계산
3. optimizer.zero_grad() : 갱신할 Variable들에 대한 모든 변화도를 0으로 만듬
4. loss.backward() : 역전파 단계 실행. 모델의 Variable들에 대한 손실의 변화도를 계산합니다.
5. optimizer.step() : 가중치 갱신.

In [14]:
EPOCHS = 20
for epoch in range(EPOCHS):
    epoch_loss = 0
    epoch_correct = 0
    net.train()

    for i, data in enumerate(traindataloader):
        inputs, labels = data
        inputs = torch.tensor(inputs, device=device).float()
        labels = torch.tensor(labels, device=device)

        # 모든 model parameter의 gradient를 0으로 설정한다.
        optimizer.zero_grad()
        # print(type(inputs), type(labels), type(outputs))
        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(inputs), type(labels), type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        epoch_loss += loss.item()

    print("Epoch : {} loss: {}".format(epoch, epoch_loss))
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Epoch : 0 loss: 42.13289733044803
Epoch : 1 loss: 20.641383207403123
Epoch : 2 loss: 21.71867738198489
Epoch : 3 loss: 16.519353855401278
Epoch : 4 loss: 14.666488809511065
Epoch : 5 loss: 13.172440581955016
Epoch : 6 loss: 12.581996018067002
Epoch : 7 loss: 10.584817011840641
Epoch : 8 loss: 9.171987907961011
Epoch : 9 loss: 10.823007059283555
Epoch : 10 loss: 7.91460071131587
Epoch : 11 loss: 8.524560377933085
Epoch : 12 loss: 8.858328242786229
Epoch : 13 loss: 8.225351545959711
Epoch : 14 loss: 8.788008000701666
Epoch : 15 loss: 9.25416670087725
Epoch : 16 loss: 8.903053456917405
Epoch : 17 loss: 8.816635721363127
Epoch : 18 loss: 6.943080166354775
Epoch : 19 loss: 7.9526237975806
Finished Training


## test 기본구조
1. model.eval() 를 통해 Test 과정이라고 내부적으로 알려줌



In [15]:
correct = 0
total = 0
net.eval()  # 모델을 eval(test) mode로 변경한다. Dropout이나 BatchNormalization을 쓰는 모델은 학습시킬 때와 평가할 때 구조/역할이 다르기 때문에 반드시 이를 명시하도록 한다.
preds = []
with torch.no_grad():
    for data in testdataloader:   # test 데이터를 불러오고 
        inputs = data
        inputs = torch.tensor(inputs, device=device).float()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)   # torch.max() : 텐서 배열의 최대 값이 들어있는 index를 리턴하는 함수.
        for pred in predicted:
          preds.append(pred.cpu().numpy()) 
          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [16]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [17]:
preds = np.array(preds)
print(preds.shape)

(18000,)


In [18]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [0]:
sample_submission.to_csv("torch_submission.csv", index=False)